<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# Create your own Chatbot 🤙

Welcome!

**ChatGPT Plus has paused signups, so we wanted to share how you can easily run your own chat GPT.**

In this notebook, we will create a chatbot using [**Zephyr 7B**](https://huggingface.co/papers/2310.16944), a popular chat model which was fine-tuned on [Mistral 7B](https://huggingface.co/mistralai/Mistral-7B-v0.1). Zephyr 7B Beta, made by the [Hugging Face H4 team](https://huggingface.co/HuggingFaceH4), **outperforms GPT 3.5, LLama 2 70B (& smaller), and all other 7B models** [[Alpaca Leaderboard](https://tatsu-lab.github.io/alpaca_eval/)]. Plus, it's **25x smaller than GPT 3.5**!

You'll be able to swap out the model if you'd like with another chat/instruct model.

### Help us make this tutorial better! Please provide feedback on the [Discord channel](https://discord.gg/y9428NwTh3) or on [X](https://x.com/harperscarroll).

A note about running Jupyter Notebooks: Press Shift + Enter to run a cell. A * in the left-hand cell box means the cell is running. A number means it has completed. If your Notebook is acting weird, you can interrupt a too-long process by interrupting the kernel (Kernel tab -> Interrupt Kernel) or even restarting the kernel (Kernel tab -> Restart Kernel). Note restarting the kernel will require you to run everything from the beginning.

If you ever have trouble importing something from Hugging Face, you may need to run `huggingface-cli login` in a shell. To open a shell in Jupyter Lab, click on 'Launcher' (or the '+' if it's not there) next to the notebook tab at the top of the screen. Under "Other", click "Terminal" and then run the command.


## Let's begin!

I used a GPU from [brev.dev](https://brev.dev). I used an A10G, with 24GB GPU Memory, 32 GB RAM, 256 GB storage. This machine is about $1/hr. 

Click the badge below to get your preconfigured instance:

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://console.brev.dev/environment/new?instance=A10G:g5.2xlarge&diskStorage=256&name=chatbot&file=https://github.com/brevdev/notebooks/raw/main/llama2-finetune.ipynb&python=3.10&cuda=12.0.1)

Once you've checked out your machine and landed in your instance page, select the specs you'd like  (I used **Python 3.10 and CUDA 12.0.1**; these should be preconfigured for you if you use the badge above) and click the "Build" button to build your verb container. Give this a few minutes.

A few minutes after your model has started Running, click the 'Notebook' button on the top right of your screen once it illuminates (you may need to refresh the screen). You will be taken to a Jupyter Lab environment, where you can upload this Notebook.

Note: You can connect your cloud credits (AWS or GCP) by clicking "Org: " on the top right, and in the panel that slides over, click "Connect AWS" or "Connect GCP" under "Connect your cloud" and follow the instructions linked to attach your credentials.

In [6]:
# You only need to run this once per machine, even if you re-start the kernel or machine
# Remove the -q to print install output
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Here we set the model. Feel free to replace it with any Hugging Face chat/instruct model (just check to use appropriate "roles" as defined below).

In [4]:
models_dict = {
    "Zephyr 7B Alpha": "HuggingFaceH4/zephyr-7b-alpha", 
    "Zephyr 7B Beta": "HuggingFaceH4/zephyr-7b-beta", 
    # You can add more chat/instruct models here 
}

model_id = models_dict["Zephyr 7B Beta"]

In [7]:
import torch
from transformers import pipeline

# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

(…)beta/resolve/main/generation_config.json: 100%|██████████| 111/111 [00:00<00:00, 694kB/s]
(…)-beta/resolve/main/tokenizer_config.json: 100%|██████████| 1.43k/1.43k [00:00<00:00, 9.26MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 330MB/s]
(…)phyr-7b-beta/resolve/main/tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 26.8MB/s]
(…)r-7b-beta/resolve/main/added_tokens.json: 100%|██████████| 42.0/42.0 [00:00<00:00, 254kB/s]
(…)eta/resolve/main/special_tokens_map.json: 100%|██████████| 168/168 [00:00<00:00, 938kB/s]


Now define the role you'd like the chatbot to assume.

In [8]:
job_description = "You are a machine learning expert. Please answer my questions."

Below we run the model! Use "stop" or "exit" to exit question answering generation. You can ignore the `UserWarning: You have modified the pretrained model configuration...` warning.

In [10]:
messages = [
    {
        "role": "system",
        "content": job_description,
    },
]
    
exit_terms = ["stop", "exit"]

while True:
    question = input("\nQuestion: ")
    if question.lower() in exit_terms: 
        break
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    response_start = output.rfind('<|assistant|>')
    print(output[response_start + len('<|assistant|>'):])


Question:  How much does it cost to train an LLM?



The cost of training a large language model (LLM) using machine learning techniques can vary significantly depending on several factors. Here are some factors that can influence the cost:

1. Hardware: Training an LLM requires a significant amount of computing power. The cost of training will depend on the type of hardware used, the size of the model, and the length of the training run. Using cloud computing services like Amazon Web Services (AWS), Google Cloud Platform (GCP), or Microsoft Azure can increase the cost, as these services charge based on usage.

2. Dataset: The size and quality of the dataset used to train the LLM can significantly affect the cost. Large datasets like the Common Crawl or the WebText dataset can require petabytes of storage and significant computing resources, which can drive up the cost.

3. Model Architecture: The specific architecture of the LLM, such as the number of layers, the type of activation functions, and the size of the model's parameters, can


Question:  Can you expand on point #4?



Sure! Preprocessing and data cleaning are crucial steps in the machine learning pipeline, especially when working with large datasets. Preprocessing involves transforming the raw data into a format that the machine learning algorithm can understand, while data cleaning involves removing any unwanted or irrelevant data, handling missing values, and dealing with outliers.

Preprocessing steps can include tasks like tokenization, which involves breaking down text into individual words or tokens, and stemming or lemmatization, which involves reducing words to their base or root forms. These steps can help the machine learning algorithm better understand the meaning of the text and make more accurate predictions.

Data cleaning is important because real-world data is often messy and contains errors, inconsistencies, and missing values. Data cleaning involves removing any duplicates, handling missing values, and dealing with outliers, which can significantly improve the accuracy and perform


Question:  Can you expand on point #6?



Certainly! After training the LLM, it's essential to evaluate its performance on a separate, unseen dataset to ensure that it can generalize well to new, previously unseen data. This process is called model evaluation, and it's an essential step in the machine learning pipeline.

Model evaluation involves measuring the LLM's performance on a separate set of data, which wasn't used during the training process. This process helps to ensure that the LLM can generalize well to new, previously unseen data and that it doesn't overfit to the training data.

There are several evaluation metrics used to measure the LLM's performance, such as accuracy, precision, recall, and F1 score. These metrics help to determine how well the LLM is able to classify or predict the correct output for a given input.

Model evaluation can be a time-consuming process, especially when dealing with large datasets. The evaluation process involves splitting the dataset into a training set, a validation set, and a te


Question:  stop


### Sweet... it worked! Epic!!!

I hope you enjoyed this tutorial on building your own Chatbot. Please join the community on [Discord](https://discord.gg/y9428NwTh3)! 

If you have any questions, please reach out to me on [X](https://x.com/harperscarroll) or in the [Discord](https://discord.gg/y9428NwTh3).

🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙